In [ ]:
!pip install mip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 76.5 MB/s eta 0:00:00


In [ ]:
instance = {
  "m" : 3,
  "n" : 7,
  "D" : [[0, 3, 3, 6, 5, 6, 6, 2],
      [3, 0, 4, 3, 4, 7, 7, 3],
      [3, 4, 0, 7, 6, 3, 5, 3],
      [6, 3, 7, 0, 3, 6, 6, 4],
      [5, 4, 6, 3, 0, 3, 3, 3],
      [6, 7, 3, 6, 3, 0, 2, 4],
      [6, 7, 5, 6, 3, 2, 0, 4],
      [2, 3, 3, 4, 3, 4, 4, 0]],
  "l" : [15, 10, 7],
  "s" : [3, 2, 6, 8, 5, 4, 4]
}

instance = {'m': 6, 'n': 8, 'D': [[0, 80, 131, 22, 41, 127, 87, 48, 113], [60, 0, 85, 82, 101, 81, 53, 106, 57], [141, 83, 0, 129, 182, 4, 64, 189, 28], [22, 82, 129, 0, 55, 125, 65, 66, 101], [41, 99, 172, 55, 0, 168, 118, 11, 154], [137, 81, 4, 125, 178, 0, 60, 185, 32], [77, 63, 64, 65, 118, 60, 0, 125, 36], [48, 126, 179, 60, 58, 175, 125, 0, 161], [113, 57, 28, 101, 154, 24, 36, 161, 0]], 'l': [200, 100, 200, 300, 200, 100], 's': [16, 25, 21, 14, 19, 14, 6, 20]}
instance

{'m': 6,
 'n': 8,
 'D': [[0, 80, 131, 22, 41, 127, 87, 48, 113],
  [60, 0, 85, 82, 101, 81, 53, 106, 57],
  [141, 83, 0, 129, 182, 4, 64, 189, 28],
  [22, 82, 129, 0, 55, 125, 65, 66, 101],
  [41, 99, 172, 55, 0, 168, 118, 11, 154],
  [137, 81, 4, 125, 178, 0, 60, 185, 32],
  [77, 63, 64, 65, 118, 60, 0, 125, 36],
  [48, 126, 179, 60, 58, 175, 125, 0, 161],
  [113, 57, 28, 101, 154, 24, 36, 161, 0]],
 'l': [200, 100, 200, 300, 200, 100],
 's': [16, 25, 21, 14, 19, 14, 6, 20]}

In [ ]:
from mip import Model, minimize, INTEGER, BINARY, xsum, quicksum, maximize
import itertools

Modello funzionante scritto male

In [ ]:
def get_package_at_time_t(time, courier, y, package_range):
  return sum(package * y[package][time][courier] for package in package_range).x


def and_model(model, A, B):
  # A and B
  # A = 0, B = 0 => 0
  # A = 0, B = 1 => 0
  # A = 1, B = 0 => 0
  # A = 1, B = 0 => 1

  # v_and new variable
  # v_and <= A
  # v_and <= B
  # v_and >= A + B - 1

  v_and = model.add_var(var_type=BINARY)

  model += v_and <= A
  model += v_and <= B
  model += v_and >= A + B - 1

  return model, v_and


def implies_model(model, A, B):
  #A => B
  # A = {0, 1}

  # If A == 1 then B == 1
  # else B == {0, 1}

  # A <= B <= 1 + A
  # B <= 1

  # A = 0 ... 0 <= B <= 1
  # A = 1 ... 1 <= B <= 2 , B <= 1 => B = 1

  model += A <= B
  model += B <= 1 + A
  model += B <= 1

  return model




def solve_multiple_couriers2(m, n, s, l, D):
    # Create the MIP model
    model = Model()

    # So that the package representing the base doens't count in the weight calculation
    s += [0]

    # Ranges
    package_range = range(n+1)
    time_range = range(n+2)
    courier_range = range(m)

    # Constant
    base_package = n
    last_time = n + 1
    coords = list(itertools.product(courier_range, time_range))


    # Variables
    # y[p][t][c] == 1 se c porta p al tempo t
    y = [[[model.add_var(name = f"y_{package_range}_{time_range}_{courier}", var_type=BINARY)
          for courier in courier_range]
          for time in time_range]
          for package in package_range]

    #y_and[p1][p2][t][c] == 1 se c porta p1 al tempo t-1 e p2 al tempo t
    y_and = [[[[model.add_var(var_type=BINARY)
                  for courier in courier_range]
                  for time in time_range]
                  for p2 in package_range]
                  for p1 in package_range]



    # Inizialze the and variable
    # If v = and(v1, v2) then we add the constraints_
    # v <= v1
    # v <= v2
    # v >= v1 + v2 - 1
    for courier in courier_range:
      for time in time_range:
        if time == 0:
          continue

        for p1 in package_range:
          for p2 in package_range:
            model += y_and[p1][p2][time][courier] <= y[p1][time - 1][courier]
            model += y_and[p1][p2][time][courier] <= y[p2][time][courier]
            model += y_and[p1][p2][time][courier] >= y[p1][time - 1][courier] + y[p2][time][courier] - 1


    distances = []
    weights = []
    for courier in courier_range:
      # weights vector, weights[c] is the amount transported by c
      weights.append(
          xsum(s[package] * y[package][time][courier] for time in time_range for package in package_range)
      )

      # distances vector, distances[c] is the amount travelled by c
      distances.append(
          xsum(
             D[p1][p2] * y_and[p1][p2][time][courier]
             for time in time_range if time != 0
             for p1 in package_range
             for p2 in package_range
          )
      )

    # Each package taken one time only
    for package in package_range:
      if package == base_package:
        continue
      model += xsum(y[package][time][courier] for time in time_range for courier in courier_range) == 1

    # A courier carry only one package at time
    for courier in courier_range:
      for time in time_range:
        model += xsum(y[package][time][courier]
                      for package in package_range) == 1

    # Every carried package must be delivered to destination and every courier must start from destination
    for courier in courier_range:
        model +=  y[base_package][0][courier] == 1
        model +=  y[base_package][last_time][courier] == 1

    # Each courier can hold packages with total weight <= max carriable weight
    for courier in courier_range:
        model += weights[courier] <= l[courier]



    #Couriers must immediately start with a package after the base
    for courier in courier_range:
      for time in time_range:
        if time == 0:
          continue

        a = y[base_package][time][courier]
        b = y[base_package][1][courier]
        model = implies_model(model, 1 - a, 1 - b)

    # Secondo metodo per implementare la stessa cosa, forse più efficiente perché indipendente dal numero di pacchi
    #for courier in courier_range:
    #  if courier < m:
    #    model += xsum(
    #      y[package][1][courier]
    #      for package in package_range if package != base_package
    #    ) == 1


    #Couriers cannot go back to the base before taking other packages
    for courier in courier_range:
      for time in time_range:
        if time == 0:
          continue

        a = y[base_package][time][courier]
        for time2 in range(time + 1, last_time):
          b = y[base_package][time2][courier]
          model = implies_model(model, a, b)

    # Calculate d_max
    d_max = model.add_var(var_type=INTEGER)
    for courier in courier_range:
      model += d_max >= distances[courier]

    model.objective = minimize(d_max)

    # Solve the MIP model
    model.optimize()

    # Extract the solution
    solution = [
        [
            int(get_package_at_time_t(time, courier, y, package_range) + 1)
            #sum(package * y[package][time][courier] for package in package_range).x
            for time in time_range
        ]   for courier in courier_range
    ]
    return model.objective_value, solution


Modello funzionante scritto bene

In [ ]:
def get_package_at_time_t(time, courier, y, package_range):
  return sum(package * y[package][time][courier] for package in package_range).x


def and_constraint(model, A, B):
  # A and B
  # A = 0, B = 0 => 0
  # A = 0, B = 1 => 0
  # A = 1, B = 0 => 0
  # A = 1, B = 0 => 1

  # v_and new variable
  # v_and <= A
  # v_and <= B
  # v_and >= A + B - 1

  v_and = model.add_var(name = f"and({A.name}, {B.name})", var_type=BINARY)

  model += v_and <= A
  model += v_and <= B
  model += v_and >= A + B - 1

  return v_and


def implies_constraint(model, A, B):
  #A => B
  # A = {0, 1}

  # If A == 1 then B == 1
  # else B == {0, 1}

  # A <= B <= 1 + A
  # B <= 1

  # A = 0 ... 0 <= B <= 1
  # A = 1 ... 1 <= B <= 2 , B <= 1 => B = 1

  model += A <= B
  model += B <= 1 + A
  model += B <= 1




def solve_multiple_couriers3(m, n, s, l, D):
    # Create the MIP model
    model = Model()

    # So that the package representing the base doens't count in the weight calculation
    s += [0]

    # Ranges
    package_range = range(n+1)
    time_range = range(n+2)
    time_range_no_zero = range(1, time_range[-1])
    courier_range = range(m)

    # Constant
    base_package = n
    last_time = n + 1
    coords = list(itertools.product(courier_range, time_range))


    # Variables
    # y[p][t][c] == 1 se c porta p al tempo t
    y = [[[model.add_var(name = f"y_{package}_{time}_{courier}", var_type=BINARY)
          for courier in courier_range]
          for time in time_range]
          for package in package_range]


    # weights vector, weights[c] is the amount transported by c
    weights = []
    for courier in courier_range:
      weights.append(
          xsum(s[package] * y[package][time][courier] for time in time_range for package in package_range)
      )


    # distances vector, distances[c] is the amount travelled by c
    distances = []
    for courier in courier_range:
      courier_distances = []
      for time in time_range_no_zero:
        for p1 in package_range:
          for p2 in package_range:
            a = y[p1][time - 1][courier]
            b = y[p2][time][courier]
            courier_distances.append(D[p1][p2] * and_constraint(model, a, b))

      distances.append(xsum(courier_distances))


    # Each package taken one time only
    for package in package_range:
      if package == base_package:
        continue
      model += xsum(y[package][time][courier] for time in time_range for courier in courier_range) == 1

    # A courier carry only one package at time
    for courier in courier_range:
      for time in time_range:
        model += xsum(y[package][time][courier] for package in package_range) == 1

    # Every carried package must be delivered to destination and every courier must start from destination
    for courier in courier_range:
        model +=  y[base_package][0][courier] == 1
        model +=  y[base_package][last_time][courier] == 1

    # Each courier can hold packages with total weight <= max carriable weight
    for courier in courier_range:
        model += weights[courier] <= l[courier]



    #Couriers must immediately start with a package after the base
    for courier in courier_range:
      for time in time_range_no_zero:
        a = y[base_package][time][courier]
        b = y[base_package][1][courier]

        # 1 - a => 1 - b is saying a != 1 => b != 1
        implies_constraint(model, 1 - a, 1 - b)

    # Secondo metodo per implementare la stessa cosa, forse più efficiente perché indipendente dal numero di pacchi
    #for courier in courier_range:
    #  if courier < m:
    #    model += xsum(
    #      y[package][1][courier]
    #      for package in package_range if package != base_package
    #    ) == 1


    #Couriers cannot go back to the base before taking other packages
    for courier in courier_range:
      for time in time_range_no_zero:
        a = y[base_package][time][courier]

        for time2 in range(time + 1, last_time):
          b = y[base_package][time2][courier]
          implies_constraint(model, a, b)

    # Calculate d_max
    d_max = model.add_var(var_type=INTEGER)
    for courier in courier_range:
      model += d_max >= distances[courier]

    model.objective = minimize(d_max)

    # Solve the MIP model
    model.optimize()

    # Extract the solution
    solution = [
        [
            int(get_package_at_time_t(time, courier, y, package_range) + 1)
            #sum(package * y[package][time][courier] for package in package_range).x
            for time in time_range
        ]   for courier in courier_range
    ]
    return model.objective_value, solution


In [ ]:

# Define the distances between couriers and delivery locations
D = [[0, 3, 3, 6, 5, 6, 6, 2],
             [3, 0, 4, 3, 4, 7, 7, 3],
             [3, 4, 0, 7, 6, 3, 5, 3],
             [6, 3, 7, 0, 3, 6, 6, 4],
             [5, 4, 6, 3, 0, 3, 3, 3],
             [6, 7, 3, 6, 3, 0, 2, 4],
             [6, 7, 5, 6, 3, 2, 0, 4],
             [2, 3, 3, 4, 3, 4, 4, 0]]

# Define the weights of each package
s = [3, 2, 6, 8, 5, 4, 4]

# Define the maximum carriable weight for each courier
l = [15, 10, 7]

# Set the number of couriers and delivery locations
m = 3
n = 7

objective_value, solution = solve_multiple_couriers3(instance['m'], instance['n'], instance['s'], instance['l'], instance['D'])
print("Objective function value:", objective_value)
#print("Solution:", solution)


for i in range(m):
  print("\n")
  t = ""
  for j in range(n+2):
    t += f"{solution[i][j]}, "
  print(t)